In [1]:
from __future__ import print_function
from __future__ import division

import tensorflow as tf
import numpy as np

import collections
from rnntest import *

In [2]:
training_data = 'datasets/pg100.txt'

training_data = read_data(training_data)

training_data = training_data[182:]
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

learning_rate = 0.001
training_iters = 2000
display_step = 100
n_input = 2

n_hidden = 256




In [3]:
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])



W = tf.Variable(tf.random_normal([n_hidden, vocab_size]))
b = tf.Variable(tf.random_normal([vocab_size]))


pred = simple_RNN(x, W, b,n_input,n_hidden)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

import random

session = tf.Session()
session.run(init)
step = 0
offset = random.randint(0,n_input+1)
end_offset = n_input + 1
acc_total = 0
loss_total = 0
with session.as_default():
    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s], offset: %d" % (symbols_in,symbols_out,symbols_out_pred,offset))
        step += 1
        offset += (n_input+1)

[<tf.Tensor 'split:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 1) dtype=float32>]
outputs[-1] Tensor("rnn/basic_lstm_cell_1/mul_2:0", shape=(?, 256), dtype=float32)
Iter= 100, Average Loss= 24.373924, Average Accuracy= 0.00%
['replacement', 'provisions'] - [of] vs [Banquo,], offset: 298
Iter= 200, Average Loss= 19.948271, Average Accuracy= 0.00%
['can', 'receive'] - [a] vs [service-], offset: 598
Iter= 300, Average Loss= 16.540838, Average Accuracy= 2.00%
['(72600.2026@compuserve.com);', 'TEL:'] - [(212-254-5093)] vs [bethought!], offset: 898
Iter= 400, Average Loss= 14.432493, Average Accuracy= 0.00%
['to', 'be'] - [new] vs [Poins?], offset: 1198
Iter= 500, Average Loss= 12.741342, Average Accuracy= 0.00%
['were', 'not'] - [summer's] vs [the], offset: 1498
Iter= 600, Average Loss= 12.703739, Average Accuracy= 0.00%
['married', 'do'] - [offend] vs [the], offset: 1798
Iter= 700, Average Loss= 12.321398, Average Accuracy= 1.00%
['thine', 'or'] - [thee.] vs [self], offs

In [4]:

with session.as_default():
    sentence = 'winter foe'
    words = sentence.split(' ')
    symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
    for i in range(32):
        keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
        onehot_pred = session.run(pred, feed_dict={x: keys})
        onehot_pred_index = int(np.argmax(onehot_pred, 1))
        sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
        symbols_in_keys = symbols_in_keys[1:]
        symbols_in_keys.append(onehot_pred_index)
    print(sentence)

winter foe unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible. unfallible.


In [5]:
print(dictionary)

{'fawn': 6623, 'considered,': 22055, 'Fame,': 17079, 'considered.': 17080, 'mustachio': 31812, 'Debts': 31813, "sir!-There's": 37719, 'fleeces': 55806, 'woods': 8558, 'sending.': 17081, 'hanging': 3896, 'offendeth': 31817, 'instant;': 31818, 'beadsmen': 31820, 'wholesome.': 56720, 'whither,': 14123, 'whither.': 22056, 'modestly,': 51216, 'scold': 14124, 'Sergeant.': 31821, 'nunnery': 31822, 'Sergeant,': 22057, 'kingdom.': 5740, 'mutinies': 17082, 'instant?': 24658, 'whither?': 17083, 'rests-that': 31824, 'out-night': 31825, "'Fiend,'": 31826, 'Retreat': 22058, 'sea-sorrow.': 31827, 'creeps.': 59889, 'wood!': 31828, 'breeders': 29547, 'slothful': 31829, 'Saba': 45640, 'spider.': 31830, "franklin's": 31831, 'spider,': 14126, 'wood,': 5811, 'chambermaids.': 31832, 'wood.': 9430, "suck'd.": 31834, 'drollery,': 49397, 'appropriation': 31835, "imagin'd": 10577, 'wood;': 12051, 'bringing': 5473, 'traitorously.': 61804, 'prizing': 31836, 'WINTER': 37725, 'fetter:': 31838, "cropp'd": 14127, 'wo